In [23]:
import numpy as np
import math
from prettytable import PrettyTable

In [17]:
#Data
#X = np.array([0,10,15,20,22.5,30])
#Y = np.array([0,227.04,362.78,517.35,602.97,901.67])
data = """0 0
10 227.04
15 362.78
20 517.35
22.5 602.97
30 901.67"""

dig = 4

In [22]:
def data_extract(data):
    data = data.replace('×10','e')
    data = data.replace('−','-')

    data = data.split("\n")
    dim = len(data)
    for i in range(dim):
        data[i] = data[i].split(' ')
        data[i] = [float(x) for x in data[i]]
    
    data = np.array(data)
    data = data.T if data.shape[0]<data.shape[1] else data
    return data

In [12]:
data = data_extract(data)
X = data[:,0]
Y = data[:,1]

In [15]:
def direct_interpolate(X,Y,x = 0, order = 1):
    n = order+1
    diff = abs(x-X)
    diff = list(enumerate(diff))
    diff.sort(key = lambda x: x[1])
    TX = []
    TY = []
    for i in range(n):
        TX.append(X[diff[i][0]])
        TY.append(Y[diff[i][0]])
    TX = np.reshape(np.array(TX),(n,1))
    TY = np.reshape(np.array(TY),(n,1))

    XX = np.ones((n,1))
    for i in range(1,n):
        XX = np.append(XX,TX**i,axis=1)

    
    print(XX)


    a = np.dot(np.linalg.inv(XX),TY)


    math_expression = 'f(t) = '
    for i in range(order,-1,-1):
        math_expression += f'({a[i][0]}) * t**{i} + ' if i != 0 else f'({a[i][0]})'

    def func(t):
        y=0
        for i in range(order+1):
            y += a[i][0]* t**i
        return y
    y = func
    return y,math_expression

    
f,s = direct_interpolate(X,Y,x=16,order=2)
print(s)
print(f(16))

[[  1.  15. 225.]
 [  1.  20. 400.]
 [  1.  10. 100.]]
f(t) = (0.3766000000000025) * t**2 + (17.733000000000004) * t**1 + (12.0499999999995)
392.1876000000002


In [21]:
def ndd(X,Y,x = 0, order = 1):
    n = order+1
    diff = abs(x-X)
    diff = list(enumerate(diff))
    diff.sort(key = lambda x: x[1])
    TX = []
    TY = {}
    for i in range(n):
        #if X[i] in 
        TX.append(X[diff[i][0]])
        TY[X[diff[i][0]]] = Y[diff[i][0]]
    
    TX.sort()
    TX = np.reshape(np.array(TX),(n,1))
    #TY = np.reshape(np.array(TY),(n,1))
    B = np.zeros((n,n))

    for i in range(n):
        for j in range(n-i):
            if i == 0:
                B[i,j] = TY[TX[j][0]]
            else:
                B[i,j] = (B[i-1,j]-B[i-1,j+1])/(TX[j][0]-TX[j+i][0])

    
    print(B)

    math_expression = 'f(t) = '
    for i in range(order,-1,-1):
        math_expression += f'({B[i][0]})'
        for j in range (i):
            math_expression += f'(t - {TX[j][0]})'
        if i != 0:
             math_expression += '+\n'

    def func(t):
        y=0
        for i in range(order+1):
            s = B[i][0]
            for j in range(i):
                s *= (t - TX[j][0])
            y += s
        return y
    y = func
    
    return y,math_expression


f,s = ndd(X,Y,x=16,order=3)
print(s)
print(f(16))


[[2.27040000e+02 3.62780000e+02 5.17350000e+02 6.02970000e+02]
 [2.71480000e+01 3.09140000e+01 4.28100000e+01 0.00000000e+00]
 [3.76600000e-01 1.69942857e+00 0.00000000e+00 0.00000000e+00]
 [1.10235714e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
f(t) = (0.1102357142857141)(t - 10.0)(t - 15.0)(t - 20.0)+
(0.37660000000000127)(t - 10.0)(t - 15.0)+
(27.147999999999996)(t - 10.0)+
(227.04)
389.5419428571429


In [20]:
def lagrange(X,Y,x = 0, order = 1):
    n = order+1
    diff = abs(x-X)
    diff = list(enumerate(diff))
    diff.sort(key = lambda x: x[1])
    TX = []
    TY = {}
    for i in range(n):
        #if X[i] in 
        TX.append(X[diff[i][0]])
        TY[X[diff[i][0]]] = Y[diff[i][0]]
    
    TX.sort()
    TX = np.reshape(np.array(TX),(n,1))

    math_expression = 'f(t) = '
    for i in range(order,-1,-1):
        math_expression += f'({TY[TX[i][0]]})'
        for j in range (n):
            if j == i:
                continue
            else:
                math_expression += f'(t - {TX[j][0]})/({TX[i][0]} - {TX[j][0]})'
        if i != 0:
             math_expression += '+\n'
    

    def func(t):
        y=0
        for i in range(n):
            s = TY[TX[i][0]]
            for j in range(n):
                if j == i:
                    continue
                else:
                    s *= ((t - TX[j][0])/(TX[i][0] - TX[j][0]))
            y += s
        return y
    y = func

    return y,math_expression


f,s = lagrange(X,Y,x=16,order=2)
print(s)
print(f(16))
    

f(t) = (517.35)(t - 10.0)/(20.0 - 10.0)(t - 15.0)/(20.0 - 15.0)+
(362.78)(t - 10.0)/(15.0 - 10.0)(t - 20.0)/(15.0 - 20.0)+
(227.04)(t - 15.0)/(10.0 - 15.0)(t - 20.0)/(10.0 - 20.0)
392.1876


In [28]:
data = """0 0
10 227.04
15 362.78
20 517.35
22.5 602.97
30 901.67"""
data = """0 6 10 13 17 20 28
6.67 17.33 42.67 37.33 30.10 29.31 28.74"""

data = data_extract(data)
X = data[:,0]
Y = data[:,1]

f,s = lagrange(X,Y,x=15,order=3)
print(s)
print(f(15))

f(t) = (29.31)(t - 10.0)/(20.0 - 10.0)(t - 13.0)/(20.0 - 13.0)(t - 17.0)/(20.0 - 17.0)+
(30.1)(t - 10.0)/(17.0 - 10.0)(t - 13.0)/(17.0 - 13.0)(t - 20.0)/(17.0 - 20.0)+
(37.33)(t - 10.0)/(13.0 - 10.0)(t - 17.0)/(13.0 - 17.0)(t - 20.0)/(13.0 - 20.0)+
(42.67)(t - 13.0)/(10.0 - 13.0)(t - 17.0)/(10.0 - 17.0)(t - 20.0)/(10.0 - 20.0)
33.281666666666666
